In [0]:
!pip3 install librosa==0.7
!pip install adversarial-robustness-toolbox
!pip install matplotlib
!pip install tensorflow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import glob
import scipy
import matplotlib.pyplot as plt
import re
import os
from scipy.io import wavfile
import librosa
import IPython.display as ipd
import librosa.display
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as keras
import art
import random

from art.attacks.evasion import FastGradientMethod 
from art.metrics import empirical_robustness
from art.metrics import clever_u
from art.attacks.evasion import SaliencyMapMethod
from art.attacks.evasion import DeepFool
from art.utils import to_categorical as one_hot
from art.metrics import loss_sensitivity as loss_sens
from art.utils import second_most_likely_class as snd_likely
from art.defences.transformer import DefensiveDistillation
from art.defences.trainer import AdversarialTrainer as AdvTrainer
import matplotlib.pyplot as plt



In [5]:
#train_df = pd.read_pickle('./drive/My Drive/adver/train_df_dec10.pkl')
#val_df = pd.read_pickle('./drive/My Drive/adver/valid_df_dec10.pkl')
x_train = np.load('./drive/My Drive/adver/x_train_vec_dec10.npy')
y_train = np.load('./drive/My Drive/adver/y_train_vec_dec10.npy')
x_test = np.load('./drive/My Drive/adver/x_test_vec_dec10.npy')
y_test = np.load('./drive/My Drive/adver/y_test_vec_dec10.npy')
print(x_train[0].shape)

def standardize(x , x_t):
  mean = np.mean(x , axis = 0 , keepdims = True)
  sd = np.std(x , axis = 0 , keepdims = True)
  return (x - mean)/sd , (x_t - mean)/sd

#x_train , x_test = standardize(x_train , x_test)


(128, 32, 1)


## MISC Functions

In [6]:
POSSIBLE_LABELS = 'yes no up down left right on off stop go unknown silence'.split(' ')
print(POSSIBLE_LABELS)
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}

def get_class_weight(y):
  ind, counts = np.unique(y, return_counts=True)
  class_weight = {i: count for i, count in zip(ind, counts)}
  class_weight = {ind: len(y)/val for ind, val in class_weight.items()}
  return class_weight

num_labels = 12

def model_cloner(src):
  model_copy= init_empty_model()
  model_copy.set_weights(src.get_weights())
  return model_copy

from sklearn.metrics import accuracy_score

def eval(cls , x , y):
  _p = cls.predict(x)
  y_pred = np.argmax(_p , axis = 1)
  return accuracy_score(y_pred , y)

def random_samplenp(x , y , sz):
  perm = np.random.permutation(len(x))
  xp = x[perm]
  yp = y[perm]
  mn = min(sz , len(x))
  return xp[:mn] , yp[:mn]

def get_avg_conf(orig_y , adv_y):
  ret = 0
  indiv = []
  for i in range(len(orig_y)):
    idx_1 = orig_y[i].argmax()
    idx_2 = adv_y[i].argmax()
    if idx_1 == idx_2:
      indiv.append(0)
      continue
    ret += adv_y[i][idx_2]
    indiv.append(adv_y[i][idx_2])
  ret /= len(orig_y)
  return (ret,indiv)

['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence']


In [0]:
import keras
from keras.layers.core import Lambda
from keras import backend as K

def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))



def init_randomized_model(conv_gamma , full_gamma):
  
  model = keras.Sequential([
      keras.layers.InputLayer(input_shape=(128,32,1)),
      keras.layers.Conv2D(64, kernel_size=(20,8), strides=1, padding='same', activation='relu'),
      PermaDropout(conv_gamma),
      keras.layers.MaxPool2D(pool_size=(2,2), padding='same'),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(64, kernel_size=(10,4),
      padding='same', activation='relu'),
      PermaDropout(conv_gamma),
      keras.layers.MaxPool2D(pool_size=(2,2), padding='same'),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(64, kernel_size=(5,2), padding='same', activation='relu'),
      PermaDropout(conv_gamma),
      keras.layers.MaxPool2D(pool_size=(2,2), padding='same'),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(64, kernel_size=(2,2), padding='same', activation='relu'),
      PermaDropout(conv_gamma),
      keras.layers.MaxPool2D(pool_size=(2,2), padding='same'),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(64, kernel_size=(2,1), padding='same', activation='relu'),
      keras.layers.MaxPool2D(pool_size=(2,2), padding='same'),
      keras.layers.BatchNormalization(),
      keras.layers.Flatten(),
      keras.layers.Dense(500, activation="relu"),
      #keras.layers.Dropout(.5),
      PermaDropout(full_gamma),
      keras.layers.Dense(250, activation="relu"),
      #keras.layers.Dropout(.4),
      PermaDropout(full_gamma),
      keras.layers.Dense(num_labels, activation="softmax")
  ])

  model.compile(loss='sparse_categorical_crossentropy', 
                optimizer='nadam',
                metrics=['accuracy'])
  return model



In [0]:
rnd_model = init_randomized_model(0.5 , 0.5)
rnd_model.fit(x_train , y_train , epochs = 20 , batch_size = 256)